In [1]:
import os

In [2]:
%pwd

'/Users/deep2.usdadiya/Desktop/DS Project/Text-Summariser-Project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/deep2.usdadiya/Desktop/DS Project/Text-Summariser-Project'

In [5]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Disable CUDA
os.environ["PYTORCH_NO_MPS"] = "1"  # Disable MPS

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [7]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [9]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

/Users/deep2.usdadiya/Desktop/DS Project/Text-Summariser-Project/textS/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-07-10 18:02:48,903: INFO: config: PyTorch version 2.3.1 available.]
[2024-07-10 18:02:48,905: INFO: config: TensorFlow version 2.16.1 available.]


In [11]:
import gc

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Disable CUDA
    torch.backends.mps.is_available = lambda: False  # Disable MPS
    device = torch.device("cpu") # type: ignore
    
    def train(self):
        device = torch.device("cpu")
        print(device)
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path) # type: ignore

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # ) 


        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500, # type: ignore
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 

        device = torch.device("cpu")

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"],  # type: ignore
                  eval_dataset=dataset_samsum_pt["validation"]) # type: ignore
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

        gc.collect()
        torch.cuda.empty_cache()

In [12]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-07-10 18:03:12,197: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-10 18:03:12,199: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-10 18:03:12,200: INFO: common: created directory at: artifacts]
[2024-07-10 18:03:12,200: INFO: common: created directory at: artifacts/model_trainer]
cpu


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/deep2.usdadiya/Desktop/DS Project/Text-Summariser-Project/textS/lib/python3.12/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  1%|          | 10/920 [09:36<13:33:58, 53.67s/it]

{'loss': 3.0679, 'grad_norm': 13.077805519104004, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}


  2%|▏         | 20/920 [3:50:24<518:45:13, 2075.01s/it]

{'loss': 2.9845, 'grad_norm': 10.075687408447266, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.02}


...

{'loss': 1.5709, 'grad_norm': 3.373673677444458, 'learning_rate': 2.3809523809523808e-06, 'epoch': 0.98}


 99%|█████████▉| 910/920 [48:03:41<10:13, 61.36s/it] 

{'loss': 1.5324, 'grad_norm': 4.052250862121582, 'learning_rate': 1.1904761904761904e-06, 'epoch': 0.99}


100%|██████████| 920/920 [48:14:27<00:00, 188.77s/it]
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


{'loss': 1.5819, 'grad_norm': 3.3042690753936768, 'learning_rate': 0.0, 'epoch': 1.0}
{'train_runtime': 173667.0787, 'train_samples_per_second': 0.085, 'train_steps_per_second': 0.005, 'train_loss': 1.8291466567827308, 'epoch': 1.0}
